In [26]:
import os
import pathlib

import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# os.environ['OMP_NUM_THREADS'] = '1'
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
tf.__version__

'2.3.1'

In [30]:
cwd = "file://"+os.getcwd()

drowsiness_dir = tf.keras.utils.get_file('drowsiness-dataset', origin=cwd+"/drowsiness-dataset.zip", extract=True)
mrl_dir = tf.keras.utils.get_file('mrl-dataset', origin=cwd+"/mrl-dataset.zip", extract=True)

print(drowsiness_dir)
batch_size = 32
img_height = 180
img_width = 180

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  drowsiness_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)



23691264/23685003 [==============================] - 0s 0us/step
/Users/henrywu/.keras/datasets/drowsiness-dataset


NotADirectoryError: [Errno 20] Not a directory: '/Users/henrywu/.keras/datasets/drowsiness-dataset'